[`pidgin.testing`](testing.md.ipynb) provides discussion and tools for __testing__ notebooks.

In [1]:
    import ast, IPython, re, functools, traitlets, typing as t, doctest, IPython, textwrap, importnb, collections, contextlib
    from importnb.utils.pytest_importnb import AlternativeModule, AlternativeSourceText

In [2]:
    from pidgin import shell, tangle
    from pidgin.applications import loaders

In [3]:
    if __name__ == '__main__':
        %reload_ext pidgin

In [7]:
    class OutputChecker(__import__('doctest').OutputChecker):
        def check_output(self, want, got, optionflags): return True if want == '...\n' else super().check_output(want, got, optionflags)

In [8]:
    @contextlib.contextmanager
    def wrapped_compiler(shell):
        def compiler(input, *args, **kwargs):
            return compile(ast.Interactive(body=shell.transform_ast(ast.parse(shell.transform_cell(textwrap.indent(input, ' '*4)))).body), *args, **kwargs)
        yield setattr(doctest, 'compile', compiler)
        try: del doctest.compile
        except: ...

In [9]:
    def run_docstring_examples(str, shell, verbose=False, compileflags=None):
        finder, runner = doctest.DocTestFinder(verbose=verbose), doctest.DocTestRunner(verbose=verbose, optionflags=doctest.ELLIPSIS)
        with wrapped_compiler(shell):
            for test in finder.find(str, name=shell.user_module.__name__): 
                test.globs = vars(shell.user_module)
                test.globs.update({**shell.user_ns, **vars(shell.user_module)})
                runner.run(test, compileflags=compileflags, clear_globs=False)
        return runner

In [10]:
    def run_test(test, shell, verbose=False, compileflags=None):
        runner = doctest.DocTestRunner(verbose=verbose, optionflags=doctest.ELLIPSIS)
        with wrapped_compiler(shell):
            test.globs = vars(shell.user_module)
            test.globs.update({**shell.user_ns, **vars(shell.user_module)})
            runner.run(test, compileflags=compileflags, clear_globs=False)
        return runner

In [11]:
    class Doctest(shell.Shell):                
        def run_cell(Formatter, text, *args, **kwargs):
            if isinstance(text, IPython.core.interactiveshell.ExecutionResult): text = text.info.raw_cell
            run_docstring_examples(text, Formatter)

In [12]:
    class PidginModule(AlternativeModule): loader = tangle.Pidgin
    class MarkdownModule(AlternativeModule): loader = loaders.MarkdownImporter
    class PidginTests(metaclass=AlternativeSourceText): modules = PidginModule, MarkdownModule
    pytest_collect_file = PidginTests.__call__